In [1]:
import os
from dotenv import load_dotenv
from nixtlats import TimeGPT
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.stattools import acf

In [2]:
load_dotenv()
os.environ['TIMEGPT_TOKEN'] = os.getenv("MY_NIXTLA_SECRET_KEY")

In [3]:
timegpt = TimeGPT(token=os.environ['TIMEGPT_TOKEN'])
timegpt.validate_token()

In [4]:
def calculate_forecasting_accuracy(y_true, y_pred):
    # Ensure numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    # Calculate errors
    errors = y_true - y_pred
    absolute_errors = np.abs(errors)
    # ME - Mean Error
    me = np.mean(errors)
    # RMSE - Root Mean Squared Error
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # MAE - Mean Absolute Error
    mae = mean_absolute_error(y_true, y_pred)
    # MPE - Mean Percentage Error
    mpe = np.mean((errors / y_true) * 100)
    # MAPE - Mean Absolute Percentage Error
    mape = np.mean(absolute_errors / y_true) * 100
    # MASE - Mean Absolute Scaled Error
    naive_forecast_errors = np.abs(np.diff(y_true))
    mase = mae / np.mean(naive_forecast_errors)
    # ACF1 - Autocorrelation of Errors at Lag 1
    acf_values = acf(errors, fft=False, missing='conservative')
    acf1 = acf_values[1] if len(acf_values) > 1 else None
    # Theil's U
    theil_u = np.sqrt(np.mean(np.square(errors))) / (np.sqrt(np.mean(np.square(y_true))) + np.sqrt(np.mean(np.square(y_pred))))
    return {
        'ME': me,
        'RMSE': rmse,
        'MAE': mae,
        'MPE': mpe,
        'MAPE': mape,
        'MASE': mase,
        'ACF1': acf1,
        'Theil\'s U': theil_u
    }

In [5]:
def plot_time_series(df_raw, df_pred, title='Time Series with Predictions'):
    # Actual values trace
    actual_trace = go.Scatter(
        x=df_raw['timestamp'],
        y=df_raw['value'],
        mode='lines',
        name='Actual'
    )
    # Predicted values trace
    prediction_trace = go.Scatter(
        x=df_pred['timestamp'],
        y=df_pred['TimeGPT'],
        mode='lines',
        name='Prediction'
    )
    
    # Layout of the plot
    layout = go.Layout(
        title=title,
        xaxis=dict(title='Date'),
        yaxis=dict(title='Value'),
        xaxis_rangeslider_visible=True  # This adds a rangeslider at the bottom
    )
    
    # Create the figure with both traces
    fig = go.Figure(data=[actual_trace, prediction_trace], layout=layout)
    
    # Show the figure
    fig.show()

In [6]:
df_ausbeer = pd.read_csv('../ausbeer.csv')
df_ausbeer['timestamp'] = pd.to_datetime(df_ausbeer['timestamp'])
df_ausbeer_train = df_ausbeer.iloc[:176,:]
df_ausbeer_test = df_ausbeer.iloc[176:,:]

In [8]:
df_ausbeer_fcst = timegpt.forecast(df_ausbeer_train, 
                 h=42,
                 freq='QS',
                 time_col='timestamp', 
                 target_col='value')

In [9]:
calculate_forecasting_accuracy(df_ausbeer_test['value'], df_ausbeer_fcst['TimeGPT'])

{'ME': -22.82361493791853,
 'RMSE': 28.468517669622386,
 'MAE': 23.755416870117195,
 'MPE': -5.556239873372521,
 'MAPE': 5.752354190012487,
 'MASE': 0.5023063907554436,
 'ACF1': -0.2204614759612863,
 "Theil's U": 0.03231758841602477}

In [10]:
plot_time_series(df_ausbeer, df_ausbeer_fcst, title='Australian Beer Production')